In [1]:
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
#machine learning and statistical methods
import statsmodels.api as sm
import math
#dataframe index manipulations
import datetime
from datetime import datetime
#selected preprocessing and evaluation methods
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import tensorflow as tf
import torch 
import pickle
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
from pytorch_lightning.callbacks import EarlyStopping
import itertools

2023-08-17 18:28:30.405912: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

In [3]:




formtemplatescategoriesPharmacy=formtemplatescategories[formtemplatescategories["metaKeywords"]=='Tax Forms']

formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formtemplates=formtemplates.drop_duplicates()
formandclone=pd.merge(formtemplates,formtemplatesclones,on="_id")
formtemplates=formtemplates.rename({"_id":"id"},axis=1)
formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
formtemplates2=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
formtemplates=df_melt._categories.apply(pd.Series) \
    .merge(formtemplates, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"id":"_id"},axis=1)
formcat=pd.concat([formtemplates2,formtemplates3])
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
 'name',
 'description',
 'metaKeywords',
 '_categories',
 '_id']]
formandclone=formandclone.loc[:,[
'_id',
 '_title',
 '_slug',
 '_description',
 '_language',
 '_featuredCategory',
 'formID',
 'form_type',
 'source',
 'date']]
formclonecategnonnanPharmacy=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.drop_duplicates(keep='first')

formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.date
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_period("D")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.rename({"date":"date2"},axis=1)
formclonecategnonnanPharmacy["day"]=formclonecategnonnanPharmacy.index.dayofweek
formclonecategnonnanPharmacy["week"]=formclonecategnonnanPharmacy.index.week
formclonecategnonnanPharmacy["dayofyear"]=formclonecategnonnanPharmacy.index.dayofyear
formclonecategnonnanPharmacy["year"]=formclonecategnonnanPharmacy.index.year
formclonecategnonnanPharmacy["quarter"]=formclonecategnonnanPharmacy.index.quarter
formclonecategnonnanPharmacy["month"]=formclonecategnonnanPharmacy.index.month
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_timestamp()






In [4]:
df=psql.sqldf("select count(_id),date,year,dayofyear from formclonecategnonnanPharmacy  group by date")

df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])


df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]





In [5]:
df2=df.copy()
df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
df2=df2.dropna()

In [6]:
tss = TimeSeriesSplit(n_splits = 5)
days = np.sort(df2.index.unique())
for train_index, test_index in tss.split(df2):
    train_days, test_days = days[train_index], days[test_index]
    X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]

In [7]:
model_parameters = pd.DataFrame(columns = ['Parameters'])

In [8]:
params_grid_neuralprophet = {
    "learning_rate":[0.01,0.1,1],
    'seasonality_mode':('multiplicative','additive'),
    'batch_size':[8,16,32],
    'weekly_seasonality':(True,False),
    "n_changepoints":[5,10,15],
    "changepoints_range":[0.1,0.5,0.8,1],
    "trend_reg":[0,0.1,0.5,1],
    "trend_reg_threshold":(True,False),
    }
grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]

In [9]:
for p in grid_neuralprophet:
    print(p)
    set_random_seed(0)
    m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
    m.add_country_holidays(country_name='US')
    m.fit(X_train, freq="D",early_stopping=True,progress="plot",validation_df=X_test)
    forecast = m.predict(X_test)
    model_parameters =model_parameters.append({'Parameters':p},ignore_index=True)




{'learning_rate': 0.01, 'seasonality_mode': 'multiplicative', 'batch_size': 8, 'weekly_seasonality': True, 'growth': 'linear', 'n_changepoints': 5, 'changepoints_range': 0.1, 'trend_reg': 0, 'trend_reg_threshold': True, 'seasonality_reg': 0}


TypeError: __init__() got an unexpected keyword argument 'petience'